In [7]:
import json
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
import pytorch_lightning as pl

In [8]:
# global variable settings

dataset_name = "jm-2"
model_name = "lstm_model_jm-2-2.pth"
filenames = [
    'unity.json',
    'rforearm.json',
    'rupperarm.json',
]

In [9]:
data = []
for i in range(len(filenames)):
    with open("./processed_data/"+dataset_name+"/"+filenames[i], 'r') as f:
        data.append(json.load(f))

# Load your sensor data
bone_data = data[0]
sensor_data1 = data[1]
sensor_data2 = data[2]

print("data loaded", len(data), len(bone_data))

data loaded 3 1818


In [10]:
class SensorData(Dataset):
    def __init__(self, features1, features2, targets, sequence_length):
        self.features1 = features1
        self.features2 = features2
        self.targets = targets
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.features1) - self.sequence_length + 1

    def __getitem__(self, idx):
        start_idx = idx
        end_idx = idx + self.sequence_length
        return (
            self.features1[start_idx:end_idx],
            self.features2[start_idx:end_idx],
            self.targets[start_idx:end_idx]
        )

In [11]:
class LSTMModel(pl.LightningModule):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(LSTMModel, self).__init__()
        self.lstm1 = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.lstm2 = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size * 2, output_size)
        self.hidden_1 = None
        self.hidden_2 = None
        self.is_inference = False

    def forward(self, x1, x2):
        output1, (h1, c1) = self.lstm1(x1, self.hidden_1 if self.is_inference else None)
        output2, (h2, c2) = self.lstm2(x2, self.hidden_2 if self.is_inference else None)
        self.hidden_1 = (h1.detach(), c1.detach())
        self.hidden_2 = (h2.detach(), c2.detach())

        output = torch.cat((output1, output2), dim=-1)
        output = self.fc(output)
        return output

    def training_step(self, batch, batch_idx):
        x1, x2, y = batch
        y_hat = self.forward(x1, x2)
        loss = nn.MSELoss()(y_hat, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x1, x2, y = batch
        y_hat = self.forward(x1, x2)
        val_loss = nn.MSELoss()(y_hat, y)
        self.log('val_loss', val_loss, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.01)

In [12]:
sequence_length = 12
batch_size = 32
epochs = 10

# Define the Dataset
data1_tensors = [torch.tensor(row) for row in sensor_data1]
data2_tensors = [torch.tensor(row) for row in sensor_data2]
bone_tensors = [torch.tensor(row) for row in bone_data]
data1_tensors = torch.stack(data1_tensors, dim=0)
data2_tensors = torch.stack(data2_tensors, dim=0)
bone_tensors = torch.stack(bone_tensors, dim=0)

dataset = SensorData(data1_tensors, data2_tensors, bone_tensors, sequence_length)
# print(dataset.shape)

# Split the dataset into training and testing sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create the dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the DataLoader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define the LSTM model
model = LSTMModel(input_size=12, hidden_size=128, output_size=35, num_layers=2)
print(model)

# Define the trainer
trainer = pl.Trainer(max_epochs=epochs)

# Train the model
trainer.fit(model, train_dataloader, val_dataloaders=test_dataloader) #接近且小

# Save the model
torch.save(model.state_dict(), "./models/" + model_name)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type   | Params
---------------------------------
0 | lstm1 | LSTM   | 204 K 
1 | lstm2 | LSTM   | 204 K 
2 | fc    | Linear | 9.0 K 
---------------------------------
418 K     Trainable params
0         Non-trainable params
418 K     Total params
1.674     Total estimated model params size (MB)


LSTMModel(
  (lstm1): LSTM(12, 128, num_layers=2, batch_first=True)
  (lstm2): LSTM(12, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=256, out_features=35, bias=True)
)


Sanity Checking: 0it [00:00, ?it/s]

C:\Users\seong\.conda\envs\GIX\lib\site-packages\pytorch_lightning\trainer\trainer.py:1613: PossibleUserWarning: The number of training batches (46) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  category=PossibleUserWarning,


Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
